
# Dominant Colour Detection 

### Author: Hasnain 
### Date: 9/12/2018

<br>
Dominant Colour Analysis
Source: https://code.likeagirl.io/finding-dominant-colour-on-an-image-b4e075f98097

Brightness of frame 
Use the dominant colour analysis but convert the image to grayscale


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2
import pickle
import copy

%matplotlib inline

In [2]:
def find_histogram(clt):
  """
  create a histogram with k clusters
  :param: clt
  :return:hist
  """
  numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
  (hist, _) = np.histogram(clt.labels_, bins=numLabels)

  hist = hist.astype("float")
  hist /= hist.sum()

  return hist

In [3]:
def plot_colors2(hist, centroids):
  bar = np.zeros((50, 300, 3), dtype="uint8")
  startX = 0

  for (percent, color) in zip(hist, centroids):
      # plot the relative percentage of each cluster
      endX = startX + (percent * 300)
      cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                    color.astype("uint8").tolist(), -1)
      startX = endX

  # return the bar chart
  return bar

In [4]:
#import movie list
movieRuntimesPath = 'Numerical Data//movie_runtimes.csv'
movieRuntimeDf = pd.read_csv(movieRuntimesPath, usecols = ['movie', 'runtime (mins)', 'effective runtime'], nrows = 9)

In [5]:
#find dominant colour through K-means clustering

def dominantColourEvaluation(movieFrames):
    dominantColourList = list()
    for img in movieFrames:
        clt = KMeans(n_clusters=1)
        clt.fit(img)
        hist = find_histogram(clt)
        bar = plot_colors2(hist, clt.cluster_centers_)
        dominantColourList.append(list(bar[0][0]))
    return dominantColourList

In [6]:
#brightness of frame
#find dominant shade through Kmeans clustering 

def dominantShadeEvaluation(movieGrayFrames):
    shadeList = list()
    for img in movieGrayFrames:
        clt = KMeans(n_clusters=1)
        clt.fit(img)
        hist = find_histogram(clt)
        bar = plot_colors2(hist, clt.cluster_centers_)
        shadeList.append(list(bar[0][0]))
    return shadeList

In [7]:
#note: frame sampling occurred at 1 frame every 3 seconds
#Read in images and perform kmeans for dominant colour/shade clustering

movieFrameNumbers = dict()
movieFrameNumbers['Hobbit 2'] = 3226
movieFrameNumbers['Buddy'] = 1817
movieFrameNumbers['Machete Kills'] = 2161
movieFrameNumbers['Walter Mitty'] = 2292
movieFrameNumbers['Paranormal Activity'] = 2022
movieFrameNumbers['The Hunger Games-Catching Fire'] = 2925
movieFrameNumbers['Star Wars-The Force Awakens'] = 2762
movieFrameNumbers['I\'m Off Then'] = 0
movieFrameNumbers['Help, I\'ve Shrunk My Teacher'] = 0

dominantColour = dict()
dominantShade = dict()
for movie in movieRuntimeDf['movie'].values:
    movieFrameList = list()
    movieGrayFrameList = list()
    for j in range(1, movieFrameNumbers[movie]+1):
        number = '{:04d}'.format(j)
        inputPath = 'Features//MovieFrames//' + movie + "//" + movie + "_" + str(number) + '.jpg'
        img = cv2.imread(inputPath)
        img = np.array(img, dtype=np.uint8)
        #convert BGR image to RGB
        img = cv2.cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #convert RGB to GRAYSCALE
        grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #reshape array from (x, y, 3) to (x*y, 3) 
        #3 rows of R G B
        img = img.reshape((img.shape[0] * img.shape[1],3)) 
        #reshape array from (x,y,1) to (x*y)    
        grayImg = grayImg.reshape((grayImg.shape[0] * grayImg.shape[1],1)) 
        #append to array
        movieFrameList.append(img)
        movieGrayFrameList.append(grayImg)
    
    print(movie)
    print('DOMINANT COLOUR')
    #movie frames both grey and colour loaded
    dominantColourList = dominantColourEvaluation(movieFrameList)
    print('DOMINANT SHADE')
    dominantShadeList = dominantShadeEvaluation(movieGrayFrameList)
    
    dominantColour[movie] = copy.deepcopy(dominantColourList)
    dominantShade[movie] = copy.deepcopy(dominantShadeList)
    print()
    

Hobbit 2
DOMINANT COLOUR
DOMINANT SHADE

Buddy
DOMINANT COLOUR
DOMINANT SHADE

Machete Kills
DOMINANT COLOUR
DOMINANT SHADE

Walter Mitty
DOMINANT COLOUR
DOMINANT SHADE

Paranormal Activity
DOMINANT COLOUR
DOMINANT SHADE

The Hunger Games-Catching Fire
DOMINANT COLOUR
DOMINANT SHADE

Star Wars-The Force Awakens
DOMINANT COLOUR
DOMINANT SHADE

I'm Off Then
DOMINANT COLOUR
DOMINANT SHADE

Help, I've Shrunk My Teacher
DOMINANT COLOUR
DOMINANT SHADE



In [8]:
pickle.dump(dominantColour, open('dominantColourOfMovies.p', 'wb'))
pickle.dump(dominantShade, open('shadeOfMovies.p', 'wb'))